In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [4]:
from tflearn.data_utils import build_hdf5_image_dataset
TW_BASE_PATH = '/Volumes/bobby/tag_walk/tag_walk/data/tag_walk'

build_hdf5_image_dataset(TW_BASE_PATH + '/images/tagged/', image_shape=(128, 128),
                         mode='folder', output_path='tagwalk.h5', categorical_labels=True)

ValueError: zero-size array to reduction operation maximum which has no identity